> Dana jest klasa:

    class Node:
        self.val = 0
        self.next = None
        
> Klasa reprezentuje węzeł jednokierunkowego łańcucha odsyłaczowego, w którym wartości $ val $ poszczególnych węzłów zostały wygenerowane zgodnie z rozkładem jednostajnym na przedziale $ [a, b] $. 

> Napisz procedurę $ sort(first) $, która sortuje taką listę. Funkcja powinna być możliwie najszybsza.

### Omówienie algorytmu

Ponieważ mamy powiedziane, że wartości zostały wygenerowane zgodnie z rozkładem jednostajnym, od razu na myśl przychodzi Bucket Sort. Jest to jedyny liniowy algorytm sortowania list odsyłaczowych, który pozwala na ich łatwe posortowanie i się idealnie do tego celu nadaje.

### Implementacja algorytmu

##### Implementacja klasy

In [1]:
class Node:
    def __init__(self, val=None):
        self.val = val
        self.next = None
        

def create_linked_list(values: 'iterable' = None) -> 'linked list head (sentinel)':
    head = Node()  # A sentinel node
    if not values: return head
    head.next = curr = Node(values[0])
    for i in range(1, len(values)):
        curr.next = Node(values[i])
        curr = curr.next
    return head


def linked_list_to_list(ll_head: 'linked list head (sentinel)') -> list:
    values = []
    curr = ll_head.next
    while curr:
        values.append(curr.val)
        curr = curr.next
    return values

##### Implementacja funkcji sortującej

In [2]:
def bucket_sort(ll_head: 'linked list head (sentinel)', min_val, max_val):
    length = linked_list_length(ll_head)
    if length < 65:
        insertion_sort(ll_head)
    else:
        k = 48
        buckets_count = length // k + 1
        # Create buckets (a list of sentinel nodes)
        buckets = []
        for _ in range(buckets_count):
            sentinel = tail = Node()
            buckets.append([sentinel, tail])
        # Calculate an interval in order to store values in proper buckets
        val_interval = (max_val - min_val) / buckets_count
        # Distribute values to the proper buckets
        curr = ll_head.next
        while curr:
            # Calculate the bucket's index depending on how much the 
            # current value is greater than the lowest one
            bucket_idx = int((curr.val - min_val) / val_interval - .5)
            buckets[bucket_idx][1].next = curr
            buckets[bucket_idx][1] = curr
            curr = curr.next
        # Sort each bucket separately
        for bucket in buckets:
            # Remove tail's next node and update a tail after sorting
            bucket[1].next = None
            bucket[1] = insertion_sort(bucket[0])
        # Link buckets together to create a sorted list
        sorted_tail = ll_head
        for bucket in buckets:
            # If a bucket is not empty, link a list to he result
            if bucket[0].next:
                sorted_tail.next = bucket[0].next
                sorted_tail = bucket[1]
            
            
def linked_list_length(ll_head: 'linked list head (sentinel)'):
    curr = ll_head.next
    length = 0
    while curr:
        length += 1
        curr = curr.next
    return length


# Inserts node in a right position maintaining the ascending order
def insert_node(ll_head: 'linked list head (sentinel)', node):
    # Insert the node before a greater one
    curr = ll_head
    while node.val > curr.next.val:
        curr = curr.next
    node.next = curr.next
    curr.next = node
        
    
def insertion_sort(ll_head: 'linked list head (sentinel)'):
    if not ll_head.next or not ll_head.next.next: return ll_head
    
    prev = ll_head.next
    while prev.next: # We start from the second node (prev.next)
        # If a current node (prev.next) has a value lower than a prev 
        # node, we have to shift this node to a right position before.
        if prev.next.val < prev.val:
            # Remove a current node
            removed = prev.next
            prev.next = prev.next.next
            # Now we insert this node in a right position
            insert_node(ll_head, removed)
        # We can skip a current node otherwise.
        else:
            prev = prev.next 
    
    return prev

###### Kilka testów

In [3]:
import random

a, b = -100, 100
random_lst = [random.randint(a, b) for _ in range(random.randint(50, 250))]
expected = sorted(random_lst)
print('Input:', random_lst)
ll_head = create_linked_list(random_lst)
bucket_sort(ll_head, a, b)
result_lst = linked_list_to_list(ll_head)
print('\nResult:', result_lst)
print('\nExpected:', expected)
print(f'\nAn algorithm is {"CORRECT" if expected == result_lst else "WRONG"}')

Input: [51, -36, 17, -84, 38, -18, 58, -43, 1, 66, -21, 5, 74, -11, 60, 87, -22, 26, 10, 35, -33, -93, -50, 91, 89, -71, -17, -65, 26, -92, -44, 28, 69, 27, 20, -44, -30, 33, 0, 23, 27, -64, -21, -90, -59, -4, 62, -34, 97, 34, -7, -92, -11, 71, -6, -7, 62, 61, 1, 0, 10, -96, -14, 33, 75, -64, 27, 8, 19, 46, -9, -74, 97, -90, 83, 13, -29, 0, -80, 72, -75, -7, -26, 29, -90, -74, 99, 79, 71, 100, -15, -83, -21, 56, -47, -12, -22, 81, -29, -89, -97, -51, 8, 50, -40, -27, -17, 33, 59, 1, 43, 42, -86, -16, 36, -99, -30, 63, 100, 29, 51, 13, -71, 42, 86, 67, -19, -37, 35, -56, 56, -15, -40, -1, 51, -94, 16, -82, -80, 27, -59, -97, -89, -67, 38, 35, -58, 4, -42, 30, -75, 54, -85, -86, 57]

Result: [-99, -97, -97, -96, -94, -93, -92, -92, -90, -90, -90, -89, -89, -86, -86, -85, -84, -83, -82, -80, -80, -75, -75, -74, -74, -71, -71, -67, -65, -64, -64, -59, -59, -58, -56, -51, -50, -47, -44, -44, -43, -42, -40, -40, -37, -36, -34, -33, -30, -30, -29, -29, -27, -26, -22, -22, -21, -21, -21, -19, 